In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten
import os
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from math import ceil
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# Model configuration
img_width, img_height = 28, 28
batch_size = 250
no_epochs = 20
no_classes = 10
validation_split = 0.2
verbosity = 1
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr



# Load MNIST dataset
(input_train, target_train), (input_test, target_test) = mnist.load_data()

# Reshape data
input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 1)
input_test = input_test.reshape(input_test.shape[0], img_width, img_height, 1)
input_shape = (img_width, img_height, 1)
    
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))


# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


Using TensorFlow backend.


In [3]:

# Fit data to model
model.fit(input_train, target_train, batch_size=batch_size,
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)


Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 4s 88us/step - loss: 0.3545 - acc: 0.8943 - val_loss: 0.0956 - val_acc: 0.9712
Epoch 2/20
48000/48000 [==============================] - 1s 29us/step - loss: 0.0949 - acc: 0.9707 - val_loss: 0.0662 - val_acc: 0.9817
Epoch 3/20
48000/48000 [==============================] - 1s 29us/step - loss: 0.0708 - acc: 0.9780 - val_loss: 0.0589 - val_acc: 0.9836
Epoch 4/20
48000/48000 [==============================] - 1s 29us/step - loss: 0.0543 - acc: 0.9825 - val_loss: 0.0500 - val_acc: 0.9861
Epoch 5/20
48000/48000 [==============================] - 1s 29us/step - loss: 0.0488 - acc: 0.9849 - val_loss: 0.0365 - val_acc: 0.9887
Epoch 6/20
48000/48000 [==============================] - 1s 29us/step - loss: 0.0392 - acc: 0.9876 - val_loss: 0.0343 - val_acc: 0.9902
Epoch 7/20
48000/48000 [==============================] - 1s 29us/step - loss: 0.0342 - acc: 0.9887 - val_loss: 0.0334 - val_acc

## Width & Height Shift

In [13]:
# specify the width an height shift arguments
width_shift_val = 0.5
height_shift_val = 0.5

# create the class object
datagen = ImageDataGenerator(width_shift_range=width_shift_val, height_shift_range=height_shift_val)

# fit the generator
datagen.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagen.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 6.393764922332764 / Test accuracy: 0.175


## Rotation

In [16]:
# specify the maximum rotation_range angle
rotation_range_val = 90

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val)

# fit the generator
datagen.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagen.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 3.4169360160827638 / Test accuracy: 0.5704


## Shear

In [21]:
# specify the shear argument
shear_range_val=85

# create the class object
datagen = ImageDataGenerator(shear_range=shear_range_val)

# fit the generator
datagen.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagen.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 2.3220121685028077 / Test accuracy: 0.6248


## Zoom

In [24]:
# specify the zoom argument
zoom_range_val=[2.5,3.5]

# create the class object
datagen = ImageDataGenerator(zoom_range=zoom_range_val)

# fit the generator
datagen.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagen.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 2.5636787059783934 / Test accuracy: 0.2356


## Rotate + W&H Shift

In [26]:
# specify the maximum rotation_range angle
rotation_range_val = 60

# specify the width and height shift arguments
width_shift_val = 0.2
height_shift_val = 0.2

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val)

# fit the generator
datagen.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagen.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 4.5558368072509765 / Test accuracy: 0.3907


## Rotate + W&H Shift + Shear

In [28]:
# specify the maximum rotation_range angle
rotation_range_val = 60

# specify the shear argument
shear_range_val = 45

# specify the width and height shift arguments
width_shift_val = 0.2
height_shift_val = 0.2

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val, shear_range=shear_range_val)

# fit the generator
datagen.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagen.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 5.234360201263428 / Test accuracy: 0.3404


## Rotate + W&H Shift + Shear + Zoom

In [30]:
# specify the maximum rotation_range angle
rotation_range_val = 60

# specify the shear argument
shear_range_val = 45

# specify the width and height shift arguments
width_shift_val = 0.2
height_shift_val = 0.2

# specify the zoom argument
zoom_range_val=[2.5,3.5]

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val, shear_range=shear_range_val, zoom_range=zoom_range_val)

# fit the generator
datagen.fit(input_test.reshape(input_test.shape[0], 28, 28, 1))

# Generate generalization metrics
score = model.evaluate(datagen.flow(input_test, target_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 3.5423620849609376 / Test accuracy: 0.1799
